In [122]:
import numpy as np
import pandas as pd
import datetime as dt
import pyomo.environ as pyo

**Source of test data**

Half-hour price data is taken from the GB day-ahead auction for 2022 sourced through N2EX. 

In [205]:
# test price curve
df = pd.read_excel('Data/auction-prices22.xls')
df = df.iloc[:,1:26]
df.rename(columns = df.iloc[4,:], inplace = True)
df.drop([0,1,2,3,4], inplace = True)
df.drop(['3B'], axis = 1, inplace = True)
df.ffill(inplace=True)
df = df.stack().reset_index().drop(columns = ['level_0','level_1']).set_axis(["Price (GBP/MWh)"], axis = 1)

hour = list(range(1,len(df)+1)) 
df.insert(loc=0, column = "Hour", value = hour)

time_range = pd.date_range(dt.datetime(2022, 1, 1), dt.datetime(2023,1,2), freq='1h')[1:]
df.set_index(time_range, inplace = True)

C:\Users\ovais\AppData\Local\Temp\ipykernel_24004\1923840718.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)


In [206]:
df.head()

,Hour,Price (GBP/MWh)
2022-01-01 01:00:00,1,115.95
2022-01-01 02:00:00,2,149.98
2022-01-01 03:00:00,3,99.99
2022-01-01 04:00:00,4,25.05
2022-01-01 05:00:00,5,7.85


**Parameters**

* t: timestep
* R_max: maximum power output/input
* S_max: maximum battery capacity
* F_charge: charge efficiency
* F_discharge: discharge efficiency
* Pt: spot price at time t
* S_t: storage at time t

**Decision variables**
* E_charge: Energy outtake at time t
* E_discharge: Energy intake at time t

**Constraints**
* St = St-1 + Eff_in*Ein - Eout/Eff_out
* for all t, St > 0
* for all t, St <= Smax
* for all t, Ein <= Rmax
* for all t, Eout <= Rmax
* for all t, Eout <= St




In [204]:
model = pyo.AbstractModel()

# battery parameters
model.T = pyo.Set(initialize = hour, ordered = True,
                  doc = 'hour')
model.Rmax = pyo.Param(initialize = 100, within = NonNegativeReals,
                      doc = 'Power capacity. Max rate of charge/discharge. (MW)')
model.Smax = pyo.Param(initialize = 200, within = NonNegativeReals,
                   doc = 'Energy capacity. Max state of charge. (MWh)')
model.F_charge = pyo.Param(initialize = 0.95, bounds = (0,1),
                       doc = 'Charging efficiency (%)')
model.F_discharge = pyo.Param(initialize = 0.95, bounds = (0,1),
                           doc = 'Discharging efficiency')
model.P = pyo.Param(initialize = df["Price (GBP/MWh)"].to_list())

# optimisation variables
model.E_charge = pyo.Var(model.T, domain = NonNegativeReals,
                     doc = "Energy charged at hour 't' (MWh)")
model.E_discharge = pyo.Var(model.T, domain = NonNegativeReals,
                        doc = "Energy discharged at hour 't' (MWh)")
model.S = pyo.Var(model.T, bounds=(0, model.Smax),
              doc = "State of charge at hour 't' (MWh)")

# battery constraints
def state_of_charge(model, t):
    """
    At hour 't', the state of charge (SOC) is the SOC from hour 't-1'
    plus the net flow of energy into the battery, accounting for efficiency loss.
    """
    if t == 1:
        return model.S[t] == 0
    else:
        return model.S[t] == (model.S[t-1]) \
                             + (model.E_charge[t-1] * model.F_charge) \
                             - (model.E_discharge[t-1] * model.F_charge)
    
model.state_of_charge = pyo.Constraint(model.T, rule = state_of_charge)
    

def charge_constraint(model, t):
    """
    The battery charges up to the power capacity for any hour t.
    """
    return model.E_charge[t] <= model.Rmax

model.charge = pyo.Constraint(model.T, rule = charge_constraint)


def discharge_constraint(model, t):
    """
    The battery discharges up to the power capacity for any hour t.
    """
    return model.E_discharge[t] <= model.Rmax

model.discharge = pyo.Constraint(model.T, rule = discharge_constraint)


#  battery constrained to not discharge when SOC is 0  
def postive_charge(model, t):
    """
    Limit discharge to amount of charge in battery, including losses
    """
    return model.E_discharge[t] <= model.S[t] * model.F_discharge


# objective function



# def objective_function(model):
#     return summation(model.)


SyntaxError: invalid syntax (2094773453.py, line 21)